In [1]:
from aiogram import Bot, Dispatcher, executor, types
from aiogram.types import ContentType
from aiogram.dispatcher.filters.state import State, StatesGroup
from aiogram.types.message import ContentTypes
from aiogram.dispatcher import FSMContext
from aiogram.contrib.fsm_storage.memory import MemoryStorage
from sqlalchemy import update
from aiogram.dispatcher.filters.state import State, StatesGroup

API_TOKEN = "5091011572:AAG4NfkC_zZjcsaAFkwLm4ZXOvhEqyLpQhY"


In [2]:
from food.paths import *
from food.search import *
import pandas as  pd
from food.psql import *
import pytz
timezones = pytz.all_timezones
import requests
from requests.structures import CaseInsensitiveDict
import urllib
from tzwhere import tzwhere

In [3]:
import nest_asyncio
nest_asyncio.apply()

In [4]:
bot = Bot(token=API_TOKEN)
storage = MemoryStorage()
dp = Dispatcher(bot, storage=storage)

dishes_table = Dishes.__table__

In [5]:
from aiogram.utils.callback_data import CallbackData
import typing

In [6]:
def geocode(q):
    geocoding_key = '5d96ac126bcb462cb373297924ab2cb4'
    url = "https://api.geoapify.com/v1/geocode/search?"

    params = {"apiKey":geocoding_key, 
              "text":q}

    resp = requests.get(url + urllib.parse.urlencode(params)).json()
    return  pd.json_normalize(resp['features']).sort_values('properties.rank.importance',ascending = False)[['properties.lat','properties.lon']].iloc[0].to_list()
   

In [7]:
def get_tz(q):
    lat,lon = geocode(q)
    return tzwhere.tzwhere().tzNameAt(lat,lon)

In [8]:
async def async_get_tz(q):
    return get_tz(q)

In [9]:
async def async_search_image(url, env='prod'):
    return search_image(url,env)

In [10]:
async def async_geocode(q):
    return geocode(q)

In [11]:
async def  async_insert_on_conflict(*args, **qwargs):
    return insert_on_conflict(*args, **qwargs)

In [12]:
async def add_sender(message):
    sender = message['from'].to_python()
    sender = pd.DataFrame(sender,index=[0]).drop(columns =['is_bot'])
    await async_insert_on_conflict(sender,'users',unique_cols=['id'])

In [13]:
def get_msg(query):
    dish = pd.read_sql(f"""select energy,protein,carb,fat from food.dishes 
                                where user_id={query['from']['id']} and 
                                message_id = {query['message']['message_id']}
                                order by id desc limit 1""",engine)
    plot_numtients = dish[['energy','protein','carb','fat']].reset_index(drop=True)
    plot_numtients.index = ['']
    return plot_numtients.astype(int).to_string()

In [14]:
def get_today_consumed(user_id):
    today_consumed = pd.read_sql(f"""select energy,grams,timestamp from {schema}.dishes
                                    where user_id = {user_id} and timestamp > now() - interval '24 hours'
                                    and grams is not null;""",engine).set_index("timestamp")
    today_consumed= today_consumed['energy']/100*today_consumed['grams']
    user_tz = engine.execute(f"""select value from food.user_properties 
                                where user_id={user_id} and
                                property='tz'
                                order by id desc limit 1""").first()

    user_tz = user_tz[0] if user_tz else 'UTC'
    today_consumed = today_consumed.tz_convert(user_tz)
    now = pd.Timestamp.now(tz = user_tz)
    today_consumed = today_consumed.reset_index()	
    this_morning = pd.Timestamp(year = now.year,month = now.month,day = now.day,hour = 3,tz = user_tz)
    today_consumed = today_consumed[today_consumed['timestamp'] > pd.Timestamp(this_morning)][0].sum()
    return int(today_consumed)

In [15]:
add_dish_cb   = CallbackData('add dish', 'action')
measurment_cb = CallbackData('measurment', 'weight')
edit_dish_cb  = CallbackData('edit_dish', 'action')

ml_version = 0.2

In [16]:
def get_keyboard(t):
    markup = types.InlineKeyboardMarkup()
    if t == 'add dish' :  
        markup.add(types.InlineKeyboardButton('add dish', callback_data=add_dish_cb.new(action='yes')))
         
    elif t == 'measurment':

        markup = types.InlineKeyboardMarkup(row_width=8)
        btns_text = tuple([(str(p)) for p in range(10,510,10)])
        markup.add(*(types.InlineKeyboardButton(text, callback_data=measurment_cb.new(weight=text)) for text in btns_text))

    elif t == 'edit_dish':

        btns_text = ('remove','edit weight','add again')
        markup.add(*(types.InlineKeyboardButton(text, callback_data=edit_dish_cb.new(action=text)) for text in btns_text))


    

    return markup 


In [17]:
#photo recieved
@dp.message_handler(content_types=ContentType.PHOTO)
async def process_photo(message: types.Message):
    await types.ChatActions.typing()

    await add_sender(message)

    photo  = message['photo'][-1]
    await photo.download(reference_images_path/photo['file_id'])
    image_url = await photo.get_url()
    dish = await async_search_image(url=image_url, env='prod')

    dish['photo_id']         = photo['file_id']
    dish['photo_message_id'] = message['message_id']
    sender = message['from'].to_python()
    dish['user_id'] = sender['id']
    dish['ml_version'] = ml_version 
    dish['timestamp']=pd.Timestamp.utcnow()

    
    plot_numtients = dish[['energy','protein','carb','fat']].reset_index(drop=True)
    plot_numtients.index = ['']
    
    msg = plot_numtients.astype(int).to_string()
    global m 
    m = message

    reply_message = await message.reply(msg, reply_markup=get_keyboard('add dish'))
    dish['message_id'] = reply_message['message_id']
    
    dish.to_sql('dishes',schema = schema,if_exists = 'append',index = False,con=engine)



   

In [18]:
async def measurment(query: types.CallbackQuery, callback_data: typing.Dict[str, str]):
    await query.answer()

    msg = query.to_python()['message']['text']
    msg = msg.split('|')[0] if '|' in msg else msg
    msg = f"{msg}"

    await bot.edit_message_text(
        msg,
        query.from_user.id,
        query.message.message_id,
        reply_markup=get_keyboard('measurment'),
    )

In [19]:
#add_dish pushed
@dp.callback_query_handler(add_dish_cb.filter(action=['yes']))
async def callback_vote_action(query: types.CallbackQuery, callback_data: typing.Dict[str, str]):
    await measurment(query, callback_data)

In [20]:
#add_dish pushed
@dp.callback_query_handler(edit_dish_cb.filter(action=['edit weight']))
async def callback_vote_action(query: types.CallbackQuery, callback_data: typing.Dict[str, str]):
    await measurment(query, callback_data)

In [21]:
def get_update(query,grams):
    energy = engine.execute(f"""select energy from food.dishes 
                                    where user_id={query['from']['id']}
                                    and message_id = {query['message']['message_id']}
                                    order by id desc limit 1""").first()[0]
                                    #['reply_to_message']['message_id']
    stmt = (
                dishes_table.update()
                            .where(dishes_table.c.message_id == query['message']['message_id'])
                            .values(grams=grams)
                            .returning(dishes_table.c.id)
        )
    session.execute(stmt)
    session.commit()

    return int(energy)


In [22]:
# message_id = engine.execute("select message_id from food.dishes").first()[0]
# user_id = engine.execute("select user_id from food.dishes").first()[0]

In [23]:
# message_id = engine.execute("select message_id from food.dishes").first()[0]
# user_id = engine.execute("select user_id from food.dishes").first()[0]
# pd.read_sql(f"""select * from food.dishes 
#                                     where user_id={user_id}
#                                     and message_id = {message_id}
#                                     order by id desc limit 1""",engine)

In [24]:
#measure provided
@dp.callback_query_handler(measurment_cb.filter(weight=[str(p) for p in range(10,510,10)]))
async def callback_vote_action(query: types.CallbackQuery, callback_data: typing.Dict[str, str]):
    await query.answer()

    grams = int(callback_data['weight'])

    energy = get_update(query,grams)

    msg = query.to_python()['message']['text']
    msg = f"{msg}\n | consumed {grams} grams \n |  {int(energy/100*grams)} kcall \n today consumed {get_today_consumed(query['from']['id'])}"

    await bot.edit_message_text(
        msg,
        query.from_user.id,
        query.message.message_id,
        reply_markup=get_keyboard('edit_dish')
    )
    

In [25]:
#remove pushed
@dp.callback_query_handler(edit_dish_cb.filter(action=['remove']))
async def callback_vote_action(query: types.CallbackQuery, callback_data: typing.Dict[str, str]):
    _ = get_update(query,0)

    msg = query.to_python()['message']['text']
    msg = msg.split('|')[0] if '|' in msg else msg
    msg = f"{msg} \n | today consumed {get_today_consumed(query['from']['id'])}"

    await query.answer()
    await bot.edit_message_text(
        msg,
        query.from_user.id,
        query.message.message_id,
        reply_markup=get_keyboard('add dish'))

In [26]:
#add again pushed
@dp.callback_query_handler(edit_dish_cb.filter(action=['add again']))
async def callback_vote_action(query: types.CallbackQuery, callback_data: typing.Dict[str, str]):

    dish = pd.read_sql(f"""select description,energy,protein,carb,fat,score,photo_id,user_id,ml_version,photo_message_id
                from food.dishes 
                                                    where user_id={query['from']['id']}
                                                    and message_id = {query['message']['message_id']} limit 1""",engine)
    dish['timestamp'] = pd.Timestamp.utcnow()

    

    msg = query.to_python()['message']['text']
    msg = msg.split('|')[0] if '|' in msg else msg
    msg = f"{msg} \n | today consumed {get_today_consumed(query['from']['id'])}"

    await query.answer()
    message = await bot.send_message(chat_id=query['from']['id'], 
                                     reply_to_message_id = dish['photo_message_id'].iloc[0],
                                     text=msg, 
                                     reply_markup=get_keyboard('add dish'))

    dish['message_id'] = message['message_id']
    dish.to_sql('dishes',schema = schema,if_exists = 'append',index = False,con=engine)
    
    

In [27]:

if __name__ == '__main__':
    executor.start_polling(dp)

/home/dima/anaconda3/envs/food/lib/python3.9/site-packages/aiogram/types/mixins.py:40: DeprecationWarning: destination parameter is deprecated, please use destination_dir or destination_file.
  warn_deprecated(
/home/dima/food_main/food/search.py:48: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811803361/work/torch/csrc/utils/tensor_new.cpp:201.)
  series2tensor = lambda series:torch.tensor([np.array(c) for c in series.values])


0.17148414695471065
0.19237631182613169
0.20882633401947048
0.21279123909309625
0.21279123909309625


In [ ]:
mm = await m.reply('test')

In [32]:
m

<Message {"message_id":2116,"from":{"id":78882798,"is_bot":false,"first_name":"Dmitry","last_name":"Grankin","username":"dmitry_grankin","language_code":"en"},"chat":{"id":78882798,"first_name":"Dmitry","last_name":"Grankin","username":"dmitry_grankin","type":"private"},"date":1655411280,"photo":[{"file_id":"AgACAgIAAxkBAAIIRGKrklA8t-HwzfBXg4mYWd_2KOFrAAKUzTEbJNJYSXjHGWHFjkpJAQADAgADcwADJAQ","file_unique_id":"AQADlM0xGyTSWEl4","file_size":1964,"width":79,"height":90},{"file_id":"AgACAgIAAxkBAAIIRGKrklA8t-HwzfBXg4mYWd_2KOFrAAKUzTEbJNJYSXjHGWHFjkpJAQADAgADbQADJAQ","file_unique_id":"AQADlM0xGyTSWEly","file_size":23078,"width":281,"height":320},{"file_id":"AgACAgIAAxkBAAIIRGKrklA8t-HwzfBXg4mYWd_2KOFrAAKUzTEbJNJYSXjHGWHFjkpJAQADAgADeAADJAQ","file_unique_id":"AQADlM0xGyTSWEl9","file_size":87197,"width":702,"height":800},{"file_id":"AgACAgIAAxkBAAIIRGKrklA8t-HwzfBXg4mYWd_2KOFrAAKUzTEbJNJYSXjHGWHFjkpJAQADAgADeQADJAQ","file_unique_id":"AQADlM0xGyTSWEl-","file_size":126861,"width":1124,"height":

In [31]:
mm['message_id']

<Message {"message_id":2119,"from":{"id":5091011572,"is_bot":true,"first_name":"calorie_bot","username":"food_classifier_bot"},"chat":{"id":78882798,"first_name":"Dmitry","last_name":"Grankin","username":"dmitry_grankin","type":"private"},"date":1655411328,"reply_to_message":{"message_id":2116,"from":{"id":78882798,"is_bot":false,"first_name":"Dmitry","last_name":"Grankin","username":"dmitry_grankin","language_code":"en"},"chat":{"id":78882798,"first_name":"Dmitry","last_name":"Grankin","username":"dmitry_grankin","type":"private"},"date":1655411280,"photo":[{"file_id":"AgACAgIAAxkBAAIIRGKrklA8t-HwzfBXg4mYWd_2KOFrAAKUzTEbJNJYSXjHGWHFjkpJAQADAgADcwADJAQ","file_unique_id":"AQADlM0xGyTSWEl4","file_size":1964,"width":79,"height":90},{"file_id":"AgACAgIAAxkBAAIIRGKrklA8t-HwzfBXg4mYWd_2KOFrAAKUzTEbJNJYSXjHGWHFjkpJAQADAgADbQADJAQ","file_unique_id":"AQADlM0xGyTSWEly","file_size":23078,"width":281,"height":320},{"file_id":"AgACAgIAAxkBAAIIRGKrklA8t-HwzfBXg4mYWd_2KOFrAAKUzTEbJNJYSXjHGWHFjkpJAQAD

In [29]:
pd.read_sql('select * from food.dishes limit 5',engine)

,id,description,energy,protein,carb,fat,score,photo_id,user_id,message_id,timestamp,ml_version,grams
0,1,"Hamburger, from school cafeteria. Potato, hash...",201.909091,11.110909,24.581818,6.775455,0.212791,AgACAgIAAxkBAAIHoGKo4JvIy0yZtyMqRy_kflZjLXFTAA...,78882798,1952,2022-06-14 19:25:17.629875+00:00,0.2,NaN
1,2,"Hamburger, on wheat bun, 1 small patty. Chicke...",246.600000,19.216000,6.735000,15.451000,0.219681,AgACAgIAAxkBAAIHo2Ko4U0m2CDWxcOktLpyNjVtl02_AA...,78882798,1955,2022-06-14 19:28:15.164676+00:00,0.2,10.0
2,3,Veal goulash with vegetables excluding carrots...,83.875000,7.653750,6.907500,2.741250,0.217987,AgACAgIAAxkBAAIHtWKo7x9gxNVey15S6Yn59MI0V2E7AA...,78882798,1973,2022-06-14 20:27:13.714982+00:00,0.2,20.0
3,4,"Gumdrops. Baked beans. Cake, jelly roll",326.222222,1.513333,75.014444,2.341111,0.214891,AgACAgIAAxkBAAIH_GKqHeN8kujU3vkVYRA-ZB1iNfJ2AA...,78882798,2044,2022-06-15 17:59:01.162326+00:00,0.2,NaN
4,5,"Gumdrops. Baked beans. Cake, jelly roll",326.222222,1.513333,75.014444,2.341111,0.214891,AgACAgIAAxkBAAIH_WKqHhvkMgv_XaY7PmiD8dtSUMHgAA...,78882798,2045,2022-06-15 17:59:57.751908+00:00,0.2,NaN


In [36]:
get_today_colsumed(78882798)

1140

In [22]:
m['message']['message_id']

2074

In [26]:
q.to_python()['message']['reply_to_message']['message_id']

2093

In [31]:
query = q

In [30]:
q['from']['id']

78882798

In [32]:
query['message']['reply_to_message']['message_id']

2093

In [35]:
grams = 50
last_photo_id,energy = engine.execute(f"""select photo_id,energy from {schema}.dishes 
                                where user_id={query['from']['id']}
                                and message_id = {query['message']['reply_to_message']['message_id']}
                                order by id desc limit 1""").first()
stmt = (
            dishes_table.update()
                        .where(dishes_table.c.photo_id == last_photo_id)
                        .values(grams=grams)
                        .returning(dishes_table.c.id)
    )
session.execute(stmt)
session.commit()
energy/100*grams

2077

In [53]:
q.to_python()['message']['text']

'energy  protein  carb  fat\n     201       11    24    6'

In [46]:
q.to_python()['message']

{'id': '338799038755595994',
 'from': {'id': 78882798,
  'is_bot': False,
  'first_name': 'Dmitry',
  'last_name': 'Grankin',
  'username': 'dmitry_grankin',
  'language_code': 'en'},
 'message': {'message_id': 2078,
  'from': {'id': 5091011572,
   'is_bot': True,
   'first_name': 'calorie_bot',
   'username': 'food_classifier_bot'},
  'chat': {'id': 78882798,
   'first_name': 'Dmitry',
   'last_name': 'Grankin',
   'username': 'dmitry_grankin',
   'type': 'private'},
  'date': 1655324353,
  'reply_to_message': {'message_id': 2077,
   'from': {'id': 78882798,
    'is_bot': False,
    'first_name': 'Dmitry',
    'last_name': 'Grankin',
    'username': 'dmitry_grankin',
    'language_code': 'en'},
   'chat': {'id': 78882798,
    'first_name': 'Dmitry',
    'last_name': 'Grankin',
    'username': 'dmitry_grankin',
    'type': 'private'},
   'date': 1655324340,
   'photo': [{'file_id': 'AgACAgIAAxkBAAIIHWKqPrT2dDD0TOJ7oMZyVeyF-kLrAALluzEb2KJRSXcB5o-BD9poAQADAgADcwADJAQ',
     'file_unique_

In [21]:
m.to_python()

{'id': '338799040818714661',
 'from': {'id': 78882798,
  'is_bot': False,
  'first_name': 'Dmitry',
  'last_name': 'Grankin',
  'username': 'dmitry_grankin',
  'language_code': 'en'},
 'message': {'message_id': 2074,
  'from': {'id': 5091011572,
   'is_bot': True,
   'first_name': 'calorie_bot',
   'username': 'food_classifier_bot'},
  'chat': {'id': 78882798,
   'first_name': 'Dmitry',
   'last_name': 'Grankin',
   'username': 'dmitry_grankin',
   'type': 'private'},
  'date': 1655322763,
  'reply_to_message': {'message_id': 2073,
   'from': {'id': 78882798,
    'is_bot': False,
    'first_name': 'Dmitry',
    'last_name': 'Grankin',
    'username': 'dmitry_grankin',
    'language_code': 'en'},
   'chat': {'id': 78882798,
    'first_name': 'Dmitry',
    'last_name': 'Grankin',
    'username': 'dmitry_grankin',
    'type': 'private'},
   'date': 1655322761,
   'photo': [{'file_id': 'AgACAgIAAxkBAAIIGWKqOIm131Yxq0QoviuNF3Sw3D4VAALcuzEb2KJRSWDpWSCB7NDRAQADAgADcwADJAQ',
     'file_unique_

Cause exception while getting updates.
Traceback (most recent call last):
  File "/home/dima/anaconda3/envs/food/lib/python3.9/site-packages/aiogram/bot/api.py", line 139, in make_request
    async with session.post(url, data=req, **kwargs) as response:
  File "/home/dima/anaconda3/envs/food/lib/python3.9/site-packages/aiohttp/client.py", line 1138, in __aenter__
    self._resp = await self._coro
  File "/home/dima/anaconda3/envs/food/lib/python3.9/site-packages/aiohttp/client.py", line 559, in _request
    await resp.start(conn)
  File "/home/dima/anaconda3/envs/food/lib/python3.9/site-packages/aiohttp/client_reqrep.py", line 898, in start
    message, payload = await protocol.read()  # type: ignore[union-attr]
  File "/home/dima/anaconda3/envs/food/lib/python3.9/site-packages/aiohttp/streams.py", line 616, in read
    await self._waiter
  File "/home/dima/anaconda3/envs/food/lib/python3.9/asyncio/futures.py", line 284, in __await__
    yield self  # This tells Task to wait for comple

78882798

<Chat {"id":78882798}>

In [55]:
types.Message(message_id=2069,chat = types.Chat(id=78882798)).text

In [24]:
types.Message??

Init signature:
types.Message(
    conf: 'typing.Dict[str, typing.Any]' = None,
    **kwargs: 'typing.Any',
) -> 'None'
Source:        
class Message(base.TelegramObject):
    """
    This object represents a message.

    https://core.telegram.org/bots/api#message
    """

    message_id: base.Integer = fields.Field()
    from_user: User = fields.Field(alias="from", base=User)
    sender_chat: Chat = fields.Field(base=Chat)
    date: datetime.datetime = fields.DateTimeField()
    chat: Chat = fields.Field(base=Chat)
    forward_from: User = fields.Field(base=User)
    forward_from_chat: Chat = fields.Field(base=Chat)
    forward_from_message_id: base.Integer = fields.Field()
    forward_signature: base.String = fields.Field()
    forward_date: datetime.datetime = fields.DateTimeField()
    is_automatic_forward: base.Boolean = fields.Field()
    reply_to_message: Message = fields.Field(base="Message")
    via_bot: User = fields.Field(base=User)
    edit_date: datetime.datetime = fields